In [5]:
import requests
import ipywidgets as widgets
from IPython.display import display

In [6]:
from bqplot import pyplot as plt
import bqplot
import numpy as np

In [7]:
base_url = "http://127.0.0.1:5000/los-cangri/"
warehouse_url = base_url+'warehouse'

In [8]:
data = requests.get(warehouse_url)

In [9]:
def get_warehouse_data(wid:int):
    data = requests.get(warehouse_url+f'/{wid}')
    return data.json()

def get_warehouse_profit(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/{wid}/profit', json=perms)
    return data.json()

def get_most_racks():
    data = requests.get(base_url+'most/rack')
    return data.json()

widgets.interact(get_warehouse_data, wid=1)
# widgets.interact(get_warehouse_profit, wid=1, uid=1)

interactive(children=(IntSlider(value=1, description='wid', max=3, min=-1), Output()), _dom_classes=('widget-i…

<function __main__.get_warehouse_data(wid: int)>

In [10]:
thing = '''
a. /most/rack – Top 10 warehouses with the most racks
b. /most/incoming – Top 5 warehouses with the most incoming transactions
c. /most/deliver – Top 5 warehouses that delivers the most exchanges
d. /most/transactions – Top 3 users that made the most transactionse. /least/outgoing – Top 3 warehouses with the least outgoing transactions
f. /most/city – Top 3 warehouses’ cities with the most transactions
'''

In [11]:
main_selection = widgets.Dropdown(options = ['Local Statistics', 'Global Statistics'])
entity_selection = widgets.Dropdown(options =  ['Warehouse','User'])
statistic_choice = widgets.Dropdown(
    options = ['Most racks', 'Most Incoming',
               'Most deliver', 'Most transactions',
               'Least outgoing', 'Most city'])

racks = get_most_racks()['Warehouses']
wids = [pair['wid'] for pair in racks]
counts = [pair['count'] for pair in racks]
fig = plt.figure()
bar_chart = plt.bar(
    wids, 
    counts, 
    scales={'x':bqplot.OrdinalScale()}
)
plt.xlabel('WID')
plt.ylabel('Count')
plt.title('Warehouses with most racks')

plt.show()
    